In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
import re

[nltk_data] Downloading package punkt to /Users/wjones/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# Data Preprocessing
path = "RottenTomatoes/DataSet/train.tsv"
df = pd.read_csv(path, sep="\t") 

# remove punctuation
df['Phrase'] = df['Phrase'].str.replace(r'[^\w\s]+', '')
# remove numbers
df['Phrase'] = df['Phrase'].str.replace(r'\d+', '')
# make it all lower case
df['Phrase'] = df['Phrase'].str.lower()
# remove non-asci characters
df.Phrase.replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)
#df['Phrase'] = df['Phrase'].str.split()

df['Tokenized_text'] = df['Phrase'].apply(word_tokenize) 

df['Sentiment']=df['Sentiment'].astype(int) #convert the star_rating column to int

df= df[df['Sentiment']!=2]

#df['label']=np.where(df['Sentiment']>=4,1,0) #1-Positve,0-Negative
# convert to NumPy Array
train = df['Phrase'].to_numpy()


/var/folders/01/r024mm7x74qd70l_5q2qs0hm0000gn/T/ipykernel_2816/2084871368.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Phrase'] = df['Phrase'].str.replace(r'[^\w\s]+', '')
/var/folders/01/r024mm7x74qd70l_5q2qs0hm0000gn/T/ipykernel_2816/2084871368.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Phrase'] = df['Phrase'].str.replace(r'\d+', '')


In [4]:
df

,PhraseId,SentenceId,Phrase,Sentiment,Tokenized_text
0,1,1,a series of escapades demonstrating the adage ...,1,"[a, series, of, escapades, demonstrating, the,..."
21,22,1,good for the goose,3,"[good, for, the, goose]"
22,23,1,good,3,[good]
33,34,1,the gander some of which occasionally amuses ...,1,"[the, gander, some, of, which, occasionally, a..."
46,47,1,amuses,3,[amuses]
...,...,...,...,...,...
156047,156048,8544,quietly suggesting the sadness and obsession b...,1,"[quietly, suggesting, the, sadness, and, obses..."
156051,156052,8544,sadness and obsession,1,"[sadness, and, obsession]"
156052,156053,8544,sadness and,1,"[sadness, and]"
156056,156057,8544,forced avuncular chortles,1,"[forced, avuncular, chortles]"


In [5]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train, Y_test = train_test_split(df['Phrase'], df['Sentiment'], test_size=0.2, random_state=30)
print("Train: " ,X_train.shape,Y_train.shape,"Test: ",(X_test.shape,Y_test.shape))

Train:  (61182,) (61182,) Test:  ((15296,), (15296,))


In [6]:
X_train,X_valid,Y_train, Y_valid = train_test_split(X_train,Y_train, test_size=0.5, random_state=30)
print("Train: " ,X_train.shape,Y_train.shape,"Test: ",(X_valid.shape,Y_valid.shape))

Train:  (30591,) (30591,) Test:  ((30591,), (30591,))


In [7]:
from nltk.stem.porter import *

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

In [9]:
countvectorizer = CountVectorizer(analyzer= 'word', stop_words='english')
tfidfvectorizer = TfidfVectorizer(analyzer='word',stop_words= 'english')


In [10]:
count_wm = countvectorizer.fit_transform(X_train)
tfidf_wm = tfidfvectorizer.fit_transform(X_train)

In [11]:
count_tokens = countvectorizer.get_feature_names()
tfidf_tokens = tfidfvectorizer.get_feature_names()

/Users/wjones/opt/anaconda3/envs/py4sci/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [12]:
df_countvect = pd.DataFrame(data = count_wm.toarray(),columns = count_tokens)
df_tfidfvect = pd.DataFrame(data = tfidf_wm.toarray(),columns = tfidf_tokens)

tf_x_train = tfidfvectorizer.fit_transform(X_train)
tf_x_valid = tfidfvectorizer.transform(X_valid)



In [13]:
print(tf_x_valid)

  (0, 10630)	0.23514574481564446
  (0, 7444)	0.23916889825206192
  (0, 7221)	0.25421379442401687
  (0, 6165)	0.4008984640885048
  (0, 5899)	0.3789598395732692
  (0, 5736)	0.36918964565400303
  (0, 5692)	0.4362689286434968
  (0, 4627)	0.2684799894839538
  (0, 360)	0.34617459135447454
  (1, 12375)	1.0
  (2, 6216)	0.8887991477692279
  (2, 4618)	0.4582969287751058
  (3, 12335)	1.0
  (4, 4375)	1.0
  (5, 8139)	0.17403148468268126
  (5, 7078)	0.40076733585757945
  (5, 6655)	0.33872096564200255
  (5, 5970)	0.41179741407057174
  (5, 2963)	0.39614310979854855
  (5, 2519)	0.3813754933436585
  (5, 2260)	0.3542061384747951
  (5, 1843)	0.3113694961792016
  (6, 12825)	0.6149049278448969
  (6, 5802)	0.7886012488653958
  (7, 12937)	0.524967893352863
  :	:
  (30587, 6892)	0.7538647738253904
  (30587, 5402)	0.6570296057143795
  (30588, 11639)	0.3104935305427958
  (30588, 11398)	0.26502101588306703
  (30588, 9064)	0.1970210574025752
  (30588, 6878)	0.29980294080900827
  (30588, 6864)	0.29419488378855974
 

In [14]:

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [15]:
kmeans = KMeans(init="k-means++", n_clusters=5, random_state=0)


In [16]:
kmeans.fit(tf_x_train)
preds = kmeans.predict(tf_x_valid)

In [17]:
import matplotlib.pyplot as plt


In [18]:
#plt.scatter(tf_x_valid[:,0], tf_x_valid[:,1], c=preds)

from 

In [19]:
from sklearn.metrics import classification_report
report=classification_report(Y_valid, preds,output_dict=True)

/Users/wjones/opt/anaconda3/envs/py4sci/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/wjones/opt/anaconda3/envs/py4sci/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/wjones/opt/anaconda3/envs/py4sci/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
report

{'0': {'precision': 0.08965414319292617,
  'recall': 0.8440463645943098,
  'f1-score': 0.16209106239460372,
  'support': 2847},
 '1': {'precision': 0.352112676056338,
  'recall': 0.045816915605241454,
  'f1-score': 0.08108327252087894,
  'support': 10913},
 '2': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 0},
 '3': {'precision': 0.5243619489559165,
  'recall': 0.017251908396946566,
  'f1-score': 0.033404774222156534,
  'support': 13100},
 '4': {'precision': 0.46048109965635736,
  'recall': 0.035915304207987137,
  'f1-score': 0.06663351566384883,
  'support': 3731},
 'accuracy': 0.10666535909254356,
 'macro avg': {'precision': 0.2853219735723076,
  'recall': 0.188606098560897,
  'f1-score': 0.0686425249602976,
  'support': 30591},
 'weighted avg': {'precision': 0.41466599632615647,
  'recall': 0.10666535909254356,
  'f1-score': 0.06644265296688108,
  'support': 30591}}

In [22]:
from sklearn.svm import LinearSVC
clf = LinearSVC(random_state=0)
clf.fit(tf_x_train,Y_train)
y_test_pred=clf.predict(tf_x_valid)

report=classification_report(Y_valid, y_test_pred,output_dict=True)
report

{'0': {'precision': 0.4579002079002079,
  'recall': 0.30944854232525465,
  'f1-score': 0.36931460909662545,
  'support': 2847},
 '1': {'precision': 0.690320874471086,
  'recall': 0.7175845322092916,
  'f1-score': 0.7036887271420227,
  'support': 10913},
 '3': {'precision': 0.6874407261888633,
  'recall': 0.7746564885496183,
  'f1-score': 0.7284473476419496,
  'support': 13100},
 '4': {'precision': 0.5048809058961343,
  'recall': 0.346555883141249,
  'f1-score': 0.41099809281627464,
  'support': 3731},
 'accuracy': 0.6587885325749403,
 'macro avg': {'precision': 0.5851356786140729,
  'recall': 0.5370613615563534,
  'f1-score': 0.5531121941742181,
  'support': 30591},
 'weighted avg': {'precision': 0.6448399126529842,
  'recall': 0.6587885325749403,
  'f1-score': 0.6474743522541285,
  'support': 30591}}

In [ ]:
x_v = tf_x_valid.toarray()

In [ ]:
from keras.utils import to_categorical

#y_target = to_categorical(train_data['Sentiment'].values)

2023-04-18 19:50:48.752492: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
y_target = to_categorical(df['Sentiment'].values)

In [ ]:
y_target.shape

(76478, 5)

In [ ]:
X_train,X_val,y_train,y_val = train_test_split(df['Phrase'], df['Sentiment'],test_size = 0.2,stratify = y_target)

In [ ]:
from tqdm import tqdm
unique_words = set()
len_max = -1

for sent in tqdm(X_train):
    unique_words.update(sent)
    if(len_max < len(sent)):
        len_max = len(sent)

print('Words in vocab : ' , len(list(unique_words)))
print('Max_length : ' , len_max)

100%|██████████| 61182/61182 [00:00<00:00, 1019040.94it/s]

Words in vocab :  27
Max_length :  265


In [ ]:
vocab_size = len(list(unique_words))
embedding_dim = 300
max_length = len_max
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

In [ ]:
tokenizer = Tokenizer(num_words = vocab_size,
                      # filters = '#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                      oov_token = oov_tok,
                      # lower = True,
                      char_level = False)

tokenizer.fit_on_texts(list(X_train))

# Training
X_train = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(X_train,
                        maxlen = max_length,
                        padding = padding_type,
                        truncating = trunc_type)

# Validation
X_val = tokenizer.texts_to_sequences(X_val)
X_val = pad_sequences(X_val,
                      maxlen = max_length,
                      padding = padding_type,
                      truncating = trunc_type)

# Testing
X_test = tokenizer.texts_to_sequences(df['Phrase'])
X_test = pad_sequences(X_test,
                       maxlen = max_length,
                       padding = padding_type,
                       truncating = trunc_type)

In [ ]:
kmeans = KMeans(init="k-means++", n_clusters=5, random_state=0)
kmeans.fit(X_train)
preds = kmeans.predict(X_val)

In [ ]:
from sklearn.metrics import classification_report
report=classification_report(y_val, preds,output_dict=True)

/Users/wjones/opt/anaconda3/envs/py4sci/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/wjones/opt/anaconda3/envs/py4sci/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/wjones/opt/anaconda3/envs/py4sci/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
report

{'0': {'precision': 0.10354477611940298,
  'recall': 0.0785007072135785,
  'f1-score': 0.08930008045052293,
  'support': 1414},
 '1': {'precision': 0.3858891288696904,
  'recall': 0.09825847846012832,
  'f1-score': 0.15663354763296317,
  'support': 5455},
 '2': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 0},
 '3': {'precision': 0.4600760456273764,
  'recall': 0.5695414515639234,
  'f1-score': 0.5089897550715787,
  'support': 6586},
 '4': {'precision': 0.11987381703470032,
  'recall': 0.10320478001086367,
  'f1-score': 0.11091652072387626,
  'support': 1841},
 'accuracy': 0.2999476987447699,
 'macro avg': {'precision': 0.21387675353023403,
  'recall': 0.1699010834496988,
  'f1-score': 0.1731679807757882,
  'support': 15296},
 'weighted avg': {'precision': 0.3597140458341907,
  'recall': 0.2999476987447699,
  'f1-score': 0.29662069545298947,
  'support': 15296}}